In [9]:
import pandas as pd
import yfinance as yf

# Load user's dataset
file_path = './FinalUnclean.csv'  # Replace with your actual file path
df = pd.read_csv(file_path)

# Get S&P 500 data from Yahoo Finance
sp500 = yf.Ticker("^GSPC")

# Download historical data (adjusted close prices) for a range of years
sp500_history = sp500.history(start="2007-01-01", end="2024-01-01", interval="1d")

# Resample to quarterly data and calculate quarterly returns
sp500_quarterly = sp500_history['Close'].resample('Q').last().pct_change()

# Create a new dataframe for quarterly returns
sp500_quarterly_df = pd.DataFrame({
    'Fiscal Quarter': sp500_quarterly.index.to_period('Q').astype(str),
    'SP500_Return': sp500_quarterly.values
})

# Adjust the format of the 'Fiscal Quarter' to match the user's dataset format
sp500_quarterly_df['Fiscal Quarter'] = sp500_quarterly_df['Fiscal Quarter'].str.replace('-', 'Q')

# Merge the S&P 500 quarterly returns with the user's dataset on 'Fiscal Quarter'
df_with_sp500 = pd.merge(df, sp500_quarterly_df, on='Fiscal Quarter', how='left')

# Save the merged dataset
df_with_sp500.to_csv('dataset_with_sp500_returns.csv', index=False)

# Optionally, display the first few rows of the merged dataset
print(df_with_sp500.head())


  CRSP Ticker Fiscal Quarter  Net Income  Operating Income  EPS Diluted  \
0         ABT         2008Q1         NaN          0.138628          NaN   
1         ABT         2008Q2    0.192314          0.180749         0.85   
2         ABT         2008Q3    0.186259          0.144658         0.69   
3         ABT         2008Q4         NaN          0.193229          NaN   
4         ABT         2009Q1    0.157889          0.214128         0.92   

   EPS Basic  CEO Dismissal  10 Year Treasury Yield  3 Month Treasury Yield  \
0        NaN              0                3.663333                2.043333   
1       0.86              0                3.886667                1.626667   
2       0.70              0                3.863333                1.493333   
3        NaN              0                3.253333                0.296667   
4       0.93              0                2.736667                0.213333   

   3 Month Treasury Yield (Bond Equivalent Basis)  ...  Inflation Rate (CP

/var/folders/_y/b_vc_rg16xdb3hx0mvp678hw0000gn/T/ipykernel_50568/1733850941.py:15: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  sp500_quarterly = sp500_history['Close'].resample('Q').last().pct_change()
/var/folders/_y/b_vc_rg16xdb3hx0mvp678hw0000gn/T/ipykernel_50568/1733850941.py:19: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  'Fiscal Quarter': sp500_quarterly.index.to_period('Q').astype(str),


In [10]:
# Load your dataset with the S&P 500 data already merged
file_path = './dataset_with_sp500_returns.csv'  # Replace with your actual file path
df = pd.read_csv(file_path)

# Calculate company growth based on Adjusted Close prices
# Assuming 'Adjusted Close' column exists in your dataset for each quarter
df['Company_Growth'] = df.groupby('CRSP Ticker')['Adjusted Close'].pct_change()

# Calculate the difference between the company's growth and the S&P 500 quarterly return
df['Growth_Difference'] = df['Company_Growth'] - df['SP500_Return']

# Save the updated dataset with the new 'Growth_Difference' column
df.to_csv('dataset_with_growth_difference_column.csv', index=False)

# Optionally, display the first few rows of the updated dataset
print(df[['CRSP Ticker', 'Fiscal Quarter', 'Company_Growth', 'SP500_Return', 'Growth_Difference']].head())


  CRSP Ticker Fiscal Quarter  Company_Growth  SP500_Return  Growth_Difference
0         ABT         2008Q1             NaN     -0.099199                NaN
1         ABT         2008Q2       -0.033071     -0.032282          -0.000789
2         ABT         2008Q3        0.093897     -0.088781           0.182678
3         ABT         2008Q4       -0.066427     -0.225582           0.159155
4         ABT         2009Q1       -0.099765     -0.116668           0.016902


/var/folders/_y/b_vc_rg16xdb3hx0mvp678hw0000gn/T/ipykernel_50568/3137147076.py:7: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['Company_Growth'] = df.groupby('CRSP Ticker')['Adjusted Close'].pct_change()


In [11]:
file_path = './dataset_with_growth_difference_column.csv'  # Replace with your actual file path
df = pd.read_csv(file_path)

# Select specific columns that you want to export
# Replace these column names with the ones you want to keep
columns_to_export = ['CRSP Ticker', 'Fiscal Quarter', 'Operating Income', 'GICS Sub-Industry', 'Consumer Sentiment Index', 'GDP Growth Rate', 'Adjusted Close', 'SP500_Return', 'Company_Growth', 'Growth_Difference', 'CEO Dismissal']

# Create a new DataFrame with just the selected columns
df_selected = df[columns_to_export]

# Export the selected columns to a new CSV file
df_selected.to_csv('selected_columns_output.csv', index=False)

# Optionally, display the first few rows of the new DataFrame
print(df_selected.head())

  CRSP Ticker Fiscal Quarter  Operating Income      GICS Sub-Industry  \
0         ABT         2008Q1          0.138628  Health Care Equipment   
1         ABT         2008Q2          0.180749  Health Care Equipment   
2         ABT         2008Q3          0.144658  Health Care Equipment   
3         ABT         2008Q4          0.193229  Health Care Equipment   
4         ABT         2009Q1          0.214128  Health Care Equipment   

   Consumer Sentiment Index  GDP Growth Rate  Adjusted Close  SP500_Return  \
0                 72.900000             -1.7       18.284336     -0.099199   
1                       NaN              NaN       17.679647     -0.032282   
2                       NaN              NaN       19.339706     -0.088781   
3                 57.666667             -8.5       18.055019     -0.225582   
4                 58.266667             -4.5       16.253757     -0.116668   

   Company_Growth  Growth_Difference  CEO Dismissal  
0             NaN                NaN  

In [12]:
file_path = './selected_columns_output.csv'  # Replace with the actual file path
df = pd.read_csv(file_path)

# Extract the year from the Fiscal Quarter (assuming the format is 'YYYYQX', e.g., '2008Q1')
df['Year'] = df['Fiscal Quarter'].str[:4]

# Select columns to interpolate
cols_to_impute = ['Consumer Sentiment Index', 'GDP Growth Rate']

# Function to interpolate only between the first and last known values within each group (ticker-year)
def interpolate_group(group):
    for col in cols_to_impute:
        # Get the first and last valid (non-NaN) index for each column
        first_valid_idx = group[col].first_valid_index()
        last_valid_idx = group[col].last_valid_index()

        # Only apply interpolation between the first and last valid values
        if first_valid_idx is not None and last_valid_idx is not None and first_valid_idx != last_valid_idx:
            group[col] = group[col].loc[first_valid_idx:last_valid_idx].interpolate(method='linear')

    return group

# Apply the interpolation function within each year group, grouped by 'CRSP Ticker' and 'Year'
df = df.groupby(['CRSP Ticker', 'Year']).apply(interpolate_group)

# Reset the index to remove 'CRSP Ticker' and 'Year' from being part of the index
df = df.reset_index(drop=True)

# Sort the data by 'CRSP Ticker' and 'Fiscal Quarter' for proper ordering
df = df.sort_values(by=['CRSP Ticker', 'Fiscal Quarter'])

# Drop the 'Year' column if you don't need it anymore
df.drop(columns='Year', inplace=True)

# Save the updated dataset with interpolated values
df.to_csv('interpolated_dataset.csv', index=False)

# Optionally, display the updated dataframe to verify the result
print(df[['CRSP Ticker', 'Fiscal Quarter', 'Consumer Sentiment Index', 'GDP Growth Rate']].head(20))

   CRSP Ticker Fiscal Quarter  Consumer Sentiment Index  GDP Growth Rate
0          ABT         2008Q1                 72.900000        -1.700000
1          ABT         2008Q2                 67.822222        -3.966667
2          ABT         2008Q3                 62.744444        -6.233333
3          ABT         2008Q4                 57.666667        -8.500000
4          ABT         2009Q1                 58.266667        -4.500000
5          ABT         2009Q2                 62.233333        -1.533333
6          ABT         2009Q3                 66.200000         1.433333
7          ABT         2009Q4                 70.166667         4.400000
8          ABT         2010Q1                 73.866667         1.900000
9          ABT         2010Q2                 73.000000         1.966667
10         ABT         2010Q3                 72.133333         2.033333
11         ABT         2010Q4                 71.266667         2.100000
12         ABT         2011Q1                 73.06

/var/folders/_y/b_vc_rg16xdb3hx0mvp678hw0000gn/T/ipykernel_50568/3947286779.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['CRSP Ticker', 'Year']).apply(interpolate_group)


In [13]:
df.dropna(inplace=True)
df.to_csv('final_data_no_NA.csv', index=False)

In [3]:
import pandas as pd
import yfinance as yf
import datetime

# Load your dataset
file_path = './final_data_no_NA.csv'  # Replace with your actual file path
df = pd.read_csv(file_path)

# Initialize a new column for Market Cap
df['Market_Cap'] = None

# Convert 'Fiscal Quarter' to date for proper lookup
def fiscal_quarter_to_date(fiscal_quarter):
    year, quarter = int(fiscal_quarter[:4]), int(fiscal_quarter[-1])
    month = 3 * quarter  # Convert quarter to month (Q1: March, Q2: June, Q3: Sept, Q4: Dec)
    return datetime.datetime(year, month, 1)

df['Date'] = df['Fiscal Quarter'].apply(fiscal_quarter_to_date)

# Fetch Market Cap data
for ticker in df['CRSP Ticker'].unique():
    # Use yfinance to download data
    stock = yf.Ticker(ticker)
    
    # Get shares outstanding and history data
    history_data = stock.history(period="max")
    history_data.index = history_data.index.tz_localize(None)  # Remove timezone
    
    shares_outstanding = stock.info.get('sharesOutstanding', None)

    # Only proceed if we have shares outstanding data
    if shares_outstanding:
        # Iterate through the rows for the current ticker
        for index, row in df[df['CRSP Ticker'] == ticker].iterrows():
            date = row['Date']
            
            # Find the closest date in the stock history manually
            closest_date_index = history_data.index.get_indexer([pd.Timestamp(date)], method="nearest")[0]
            adjusted_close = history_data['Close'].iloc[closest_date_index]

            # Calculate market cap as Adjusted Close * Shares Outstanding
            market_cap = adjusted_close * shares_outstanding
            df.at[index, 'Market_Cap'] = market_cap
    else:
        print(f"Shares outstanding data not available for {ticker}")

# Replace 'Operating Income' column with 'Market Cap'
df['Operating Income'] = df['Market_Cap']
df.drop(columns=['Market_Cap', 'Date'], inplace=True)

# Save the updated dataset
df.to_csv('dataset_with_market_cap.csv', index=False)

# Display the first few rows of the updated dataset
print(df.head())

  CRSP Ticker Fiscal Quarter    Operating Income      GICS Sub-Industry  \
0         ABT         2008Q2   31755477786.42041  Health Care Equipment   
1         ABT         2008Q3   33867233689.63916  Health Care Equipment   
2         ABT         2008Q4  28989964685.471191  Health Care Equipment   
3         ABT         2009Q1    27389145161.5177  Health Care Equipment   
4         ABT         2009Q2  26429814301.533325  Health Care Equipment   

   Consumer Sentiment Index  GDP Growth Rate  Adjusted Close  SP500_Return  \
0                 67.822222        -3.966667       17.679647     -0.032282   
1                 62.744444        -6.233333       19.339706     -0.088781   
2                 57.666667        -8.500000       18.055019     -0.225582   
3                 58.266667        -4.500000       16.253757     -0.116668   
4                 62.233333        -1.533333       16.175817      0.152218   

   Company_Growth  Growth_Difference  CEO Dismissal  
0       -0.033071         

In [4]:
df.rename(columns={'Operating Income': 'Market Cap'}, inplace=True)
df.to_csv('final_dataset.csv', index=False)